# CSE-CIC-IDS 2017

In [4]:
model_id = "gradientboostedtrees4-multiclass-maxdepth6"

In [5]:
import numpy as np
np.random.seed(42)
np.set_printoptions(suppress=True)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import glob, pickle, time, sklearn

In [6]:
print(sklearn.__version__)

0.21.3


In [7]:
NOTEBOOK_PATH = "C:/Users/Xetrov/Desktop/SciFair20/Code/"

In [8]:
x_scaled = pd.read_csv(NOTEBOOK_PATH + "IDS2017/x_scaled_powertransform.csv")

y_df_enc = pd.read_csv(NOTEBOOK_PATH + "IDS2017/y_grouped_1henc.csv")

# Split data into train and test

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
x_train, x_valtest, y_train, y_valtest = train_test_split(x_scaled, y_df_enc, test_size = 0.4, random_state = 42)

In [11]:
x_val, x_test, y_val, y_test = train_test_split(x_valtest, y_valtest, test_size = 0.5, random_state = 42)

In [12]:
del x_train
del y_train

del x_valtest 
del y_valtest

# ADASYN

In [10]:
x_train_res = pd.read_csv(NOTEBOOK_PATH + "IDS2017/x_adasyn.csv").to_numpy()

In [11]:
y_train_res = pd.read_csv(NOTEBOOK_PATH + "IDS2017/y_adasyn.csv")

In [12]:
y_train_res = y_train_res[y_train_res.columns[0]].to_numpy()

# Train model

In [13]:
# from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier

In [16]:
clf = GradientBoostingClassifier(n_estimators=350, max_depth=6, learning_rate=0.1, subsample=1, max_features="sqrt", random_state=42, verbose=2)

In [17]:
%%time
clf.fit(x_train_res, y_train_res)

      Iter       Train Loss   Remaining Time 
         1    13810159.2664          620.38m
         2    11061352.8122          742.10m
         3     9111316.2399          787.65m
         4     7699690.1704          807.47m
         5     6533373.4773          817.05m
         6     5558012.9225          822.40m
         7     4805590.5460          828.76m
         8     4187294.0538          831.76m
         9     3662407.0713          831.91m
        10     3214292.3446          831.95m
        11     2813399.6097          831.55m
        12     2501113.0222          831.73m
        13     2227821.6290          830.11m
        14     1981928.8190          828.41m
        15     1783023.3560          827.01m
        16     1607452.0227          826.38m
        17     1436567.1599          824.24m
        18     1288212.8729          822.51m
        19     1169881.7031          819.94m
        20     1059157.5517          818.42m
        21      967039.0902          816.38m
        2

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=6,
                           max_features='sqrt', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=350,
                           n_iter_no_change=None, presort='auto',
                           random_state=42, subsample=1, tol=0.0001,
                           validation_fraction=0.1, verbose=2,
                           warm_start=False)

**Save model**

In [18]:
import pickle

In [19]:
import time

model_filename = model_id  + " [" + time.strftime("%Y%m%d %H%M") + "]"

In [21]:
save_file = open(NOTEBOOK_PATH + "Models/" + model_filename + ".pkl", "wb")
saved_model = pickle.dump(clf, save_file)
save_file.close()

**Test model**

In [21]:
pred = clf.predict(x_val)

In [22]:
pred_index = pred
y_index = list(np.argmax(y_val.to_numpy(), axis=1))

atktypes = sorted(y_val.columns)
pred_series = pd.Series(pred_index, name="Pred").replace(dict(enumerate(atktypes)))
y_series = pd.Series(y_index, name="Actual").replace(dict(enumerate(atktypes)))

matrix = pd.crosstab(pred_series, y_series, margins=True)
matrix

Actual,BENIGN,Botnet,Brute Force,DoS/DDoS,Infiltration,PortScan,Web Attack,All
Pred,,,,,,,,
BENIGN,434267,28,0,380,0,82,15,434772
Botnet,3972,369,0,0,0,0,0,4341
Brute Force,774,0,2810,0,0,58,0,3642
DoS/DDoS,8136,0,0,75953,0,13,0,84102
Infiltration,18,0,0,0,7,0,0,25
PortScan,6616,0,0,2,0,31632,0,38250
Web Attack,424,0,0,175,0,2,416,1017
All,454207,397,2810,76510,7,31787,431,566149


# F1 Score

In [26]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [25]:
pred = clf.predict(x_test)
pred_index = list(pred)

In [27]:
y_index = list(np.argmax(y_test.to_numpy(), axis=1))
f1_micro = f1_score(y_index, pred_index, average='micro')
f1_macro = f1_score(y_index, pred_index, average='macro')

print(f1_micro)
print(f1_macro)

0.998631102412969
0.9333264139871215


In [24]:
# All trees

0.9970785076013559
0.7903159491636783


In [25]:
# All trees

Actual,BENIGN,Botnet,Brute Force,DoS/DDoS,Infiltration,PortScan,Web Attack,All
Pred,,,,,,,,
BENIGN,453551,17,2,75,2,20,2,453669
Botnet,210,367,0,0,0,0,0,577
Brute Force,34,0,2746,11,0,0,0,2791
DoS/DDoS,430,0,4,75591,0,3,24,76052
Infiltration,112,0,0,11,3,2,0,128
PortScan,260,0,0,199,0,31850,32,32341
Web Attack,193,0,0,10,0,1,387,591
All,454790,384,2752,75897,5,31876,445,566149


In [29]:
atktypes = sorted(y_val.columns)
pred_series = pd.Series(pred_index, name="Pred").replace(dict(enumerate(atktypes)))
y_series = pd.Series(y_index, name="Actual").replace(dict(enumerate(atktypes)))

matrix = pd.crosstab(pred_series, y_series, margins=True)
# matrix.to_csv(NOTEBOOK_PATH + "Confusion Matrices/test_" + model_filename + ".csv")
matrix

Actual,BENIGN,Botnet,Brute Force,DoS/DDoS,Infiltration,PortScan,Web Attack,All
Pred,,,,,,,,
BENIGN,454125,14,1,2,1,7,1,454151
Botnet,253,370,0,0,0,0,0,623
Brute Force,5,0,2747,0,0,0,0,2752
DoS/DDoS,198,0,4,75870,0,1,20,76093
Infiltration,0,0,0,0,4,0,0,4
PortScan,197,0,0,22,0,31867,33,32119
Web Attack,12,0,0,3,0,1,391,407
All,454790,384,2752,75897,5,31876,445,566149


Tree selection

In [22]:
clf = pickle.load(open(NOTEBOOK_PATH + "Models/gradientboostedtrees4-multiclass-maxdepth6 [20191120 1507].pkl", "rb"))

In [15]:
dir(clf)

['_SUPPORTED_LOSS',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_check_initialized',
 '_check_params',
 '_clear_state',
 '_estimator_type',
 '_fit_stage',
 '_fit_stages',
 '_get_param_names',
 '_get_tags',
 '_init_state',
 '_is_initialized',
 '_make_estimator',
 '_raw_predict',
 '_raw_predict_init',
 '_required_parameters',
 '_resize_state',
 '_rng',
 '_staged_raw_predict',
 '_validate_estimator',
 '_validate_y',
 'alpha',
 'apply',
 'classes_',
 'criterion',
 'decision_function'

In [23]:
clf.estimators_ = clf.estimators_[:220]

In [24]:
len(clf.estimators_)

220